<a href="https://colab.research.google.com/github/hamzasabouri/Healthcare-Applications/blob/main/BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Telecharger fichier

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving symptoms_diseases_modified.csv to symptoms_diseases_modified.csv


Liste de fichier

In [ ]:
import os
os.listdir()


['.config', 'symptoms_diseases_modified.csv', 'sample_data']

Bibliotheque

In [ ]:
!pip install pyspark  gradio



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


Code de source

In [ ]:
import gradio as gr
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, CountVectorizer, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import udf, col
from pyspark.ml.linalg import Vector
from pyspark.sql.types import StringType

# Initialiser Spark
def initialize_spark():
    return SparkSession.builder.appName("Medical Recommender").master("local[*]").getOrCreate()

spark = initialize_spark()

# Charger les données
data = spark.read.option("header", "true").option("inferSchema", "true").csv("symptoms_diseases_modified.csv")

if not ("Symptoms" in data.columns and "Disease Predictions" in data.columns):
    raise ValueError("Le CSV doit contenir les colonnes 'Symptoms' et 'Disease Predictions'")

# Prétraitement et pipeline
indexer = StringIndexer(inputCol="Disease Predictions", outputCol="Label").fit(data)
tokenizer = Tokenizer(inputCol="Symptoms", outputCol="Tokens")
vectorizer = CountVectorizer(inputCol="Tokens", outputCol="Features")
classifier = LogisticRegression(labelCol="Label", featuresCol="Features", maxIter=10)

pipeline = Pipeline(stages=[indexer, tokenizer, vectorizer, classifier])
training_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(training_data)

# Liste des symptômes uniques
symptoms_list = [
"Chronic cough", "Weight loss", "Night sweats", "Fever", "High fever", "Chills", "Sweating", "Muscle aches", "Persistent fatigue", "Swollen lymph nodes", "Frequent infections", "Jaundice", "Abdominal pain", "Dark urine", "Fatigue", "Nausea", "Joint pain", "Cough", "Runny nose", "Rash", "Mild fever", "Swollen salivary glands", "Headache", "Muscle pain", "Sore throat", "Swollen neck", "Difficulty breathing", "Severe coughing fits", "Vomiting", "Muscle stiffness", "Jaw cramping", "Difficulty swallowing", "Seizures", "Severe diarrhea", "Dehydration", "Leg cramps", "Painless sore", "Skin rash", "Painful urination", "Genital discharge", "Pelvic pain", "Swollen testicles", "Lower abdominal pain", "Abnormal discharge", "Bleeding after sex", "Skin lesions", "Numbness", "Muscle weakness", "Eye problems", "Severe headache", "Stiff neck", "Sensitivity to light", "Weakness", "Constipation or diarrhea", "Red eyes", "Bleeding", "Severe joint pain", "Bleeding gums", "Itchy rash", "Loss of appetite", "Paralysis", "Body aches", "Butterfly rash", "Vision problems", "Swelling", "Stiffness", "Diarrhea", "Bloody diarrhea", "Urgent bowel movements", "Frequent urination", "Extreme thirst", "Unexplained weight loss", "Weight gain", "Cold intolerance", "Hair thinning", "Rapid heartbeat", "Bulging eyes", "Anxiety", "Drooping eyelids", "Scaly patches", "Itching", "Skin redness", "Dizziness", "Blurred vision", "Shortness of breath", "Chest pain", "Leg pain", "Swelling in legs", "Sudden weakness", "Facial drooping", "Speech difficulty", "Loss of balance", "Sudden shortness of breath", "Coughing blood", "Heart murmur", "Irregular heartbeat", "Palpitations", "Sharp chest pain", "Lump in breast", "Nipple discharge", "Breast pain", "Skin changes", "Blood in stool", "Weak urine flow", "Erectile dysfunction", "Easy bruising", "Unusual mole", "Bleeding mole", "Lump under skin", "Bone pain", "Memory loss", "Confusion", "Mood changes", "Difficulty speaking", "Tremors", "Slow movement", "Balance problems", "Loss of consciousness", "Unusual sensations", "Jerking movements", "Muscle twitching", "Light sensitivity", "Visual disturbances", "Involuntary movements", "Memory problems", "Depression", "Difficulty walking", "Tingling", "Excessive daytime sleepiness", "Cataplexy", "Sleep paralysis", "Hallucinations", "Loss of coordination", "Scoliosis", "Increased thirst", "No symptoms early on", "Xanthomas", "Heart attack risk", "Redness", "Heat", "High blood pressure", "High blood sugar", "Abdominal obesity", "High cholesterol", "Excessive body weight", "Severe flank pain", "Bloody urine", "Blood in urine", "Wheezing", "Chest tightness", "Cough with phlegm", "Clubbing of fingers", "Loss of taste/smell", "Dry cough", "Cyclical fever", "Persistent cough", "Excessive thirst", "Coughing", "Headaches (severe)", "Nosebleeds", "Throbbing headache", "Aura", "Abdominal cramps", "Right lower abdominal pain", "Bloating", "Diarrhea/constipation", "Gas", "Arm weakness", "Pain radiating to arm", "Cold sweat", "Unconsciousness", "Persistent sadness", "Loss of interest", "Sleep changes", "Excessive worry", "Restlessness", "Muscle tension", "Concentration issues", "Bullseye rash", "Itchy blisters", "Dry skin", "Morning stiffness", "Low-grade fever", "Chest discomfort", "Muscle cramps", "Facial pain", "Nasal congestion", "Postnasal drip", "Reduced mobility", "Sudden joint pain (big toe)", "Tenderness", "Red patches with scales", "Itchy skin", "Dryness", "Skin cracking", "Urgency to defecate", "Burning stomach pain", "Blotting", "Dark stools", "Sudden upper abdominal pain", "Burning urination", "Cloudy urine", "Fever (acute phase)", "Blistering sores", "Burning sensation", "Flu-like symptoms", "Koplik spots", "Jaw locking", "Hyperactivity", "Disorientation", "Balance issues", "Pale skin", "Changing mole", "Irregular borders", "Muscle spasms", "Slurred speech", "Progressive muscle weakness", "Frequent falls", "Walking difficulty", "Temporary confusion", "Staring spells", "Uncontrollable movements", "Sudden numbness", "Agitation", "Hydrophobia", "Jaw cramps", "Painful spasms", "Joint swelling", "Lower back pain", "Reduced chest expansion", "Skin thickening", "Raynaud phenomenon", "Digestive issues", "Dry eyes", "Dry mouth", "Rectal bleeding", "Diarrhea with blood", "Diarrhea or constipation", "Change in bowel habits", "Upper abdominal pain", "Abdominal pain radiating to back", "Right upper abdominal pain", "Abdominal discomfort", "Darkening skin", "Salty tasting skin", "Poor growth", "Reduced exercise tolerance", "Progressive shortness of breath", "Coughing up blood", "Foul smelling sputum", "Recurrent infections", "Loud snoring", "Daytime sleepiness", "Morning headache", "Leg swelling", "Warmth in affected area", "Leg pain while walking", "Slow healing wounds", "Pulsating feeling in abdomen", "Fluttering in chest", "Racing heartbeat", "New murmur", "Leg pain with walking", "Yellowish skin deposits", "Insulin resistance", "Intense joint pain", "Redness in joints", "Episodes of pain", "Swelling in hands and feet", "Prolonged bleeding", "Kidney problems", "Nodal skin eruptions", "Dischromic  patches", "Continuous sneezing", "Shivering", "Watering from eyes", "Stomach pain", "Acidity", "Ulcers on tongue", "Yellowish skin", "Burning micturition", "Spotting  urination", "Passage of gases", "Indigestion", "Muscle wasting", "Patches in throat", "Extra marital contacts", "Lethargy", "Irregular sugar level", "Sunken eyes", "Diarrhoea", "Breathlessness", "Family history", "Lack of concentration", "Blurred and distorted vision", "Excessive hunger", "Back pain", "Weakness in limbs", "Neck pain", "Weakness of one body side", "Altered sensorium", "Yellowing of eyes", "Swelling of stomach", "Distention of abdomen", "Constipation", "Pain during bowel movements", "Pain in anal region", "Bloody stool", "Irritation in anus", "Cramps", "Bruising", "Obesity", "Swollen legs", "Cold hands and feets", "Mood swings", "Knee pain", "Hip joint pain", "Swelling joints", "Movement stiffness", "Painful walking", "Spinning movements", "Pus filled pimples", "Blackheads", "Scurring", "Bladder discomfort", "Foul smell of urine", "Continuous feel of urine", "Skin peeling", "Silver like dusting", "Small dents in nails", "Blister", "Red sore around nose", "Yellow crust ooze"
]

# Fonction de prédiction
def predict_disease(selected_symptoms):
    if len(selected_symptoms) > 4:
        return "Veuillez sélectionner un maximum de 4 symptômes."

    symptoms_str = ", ".join(selected_symptoms)  # Transformer la liste en texte
    user_input = spark.createDataFrame([(symptoms_str,)], ["Symptoms"])
    predictions = model.transform(user_input)
    labels = indexer.labels

    index_to_label = udf(lambda index: labels[int(index)], StringType())
    final_predictions = predictions.withColumn("Disease", index_to_label(col("prediction"))).select("Symptoms", "Disease", "probability")
    predicted_row = final_predictions.collect()[0]
    probabilities = predicted_row["probability"].toArray()
    top_predictions = sorted(zip(probabilities, labels), key=lambda x: -x[0])[:3]

    return {disease: prob for prob, disease in top_predictions}

# Interface Gradio
iface = gr.Interface(
    fn=predict_disease,
    inputs=gr.Dropdown(
        choices=symptoms_list,
        label="Sélectionnez vos symptômes (maximum 4)",
        multiselect=True,  # Permet de sélectionner plusieurs symptômes
        max_choices=4     # Limite à 4 symptômes maximum
    ),
    outputs=gr.Label(num_top_classes=3),
    title="Recommander de Maladies Médicales",
    description="Sélectionnez jusqu'à 4 symptômes, puis soumettez pour prédire les maladies possibles avec leurs taux de confiance."
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57d1473bd0912fb121.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
